In [7]:
import pandas as pd
import numpy as np

In [14]:
!pip install langchain_pinecone

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 505.7 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 999.8/999.8 kB 630.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 764.4 kB/s eta 0:00:00a 0:00:01


In [15]:
import os
import pandas as pd
from dotenv import load_dotenv
from langchain.docstore.document import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore

/home/prakhar/.local/lib/python3.13/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [17]:
documents = []
for _, row in df.iterrows():
    # Create a descriptive text string from the row's data
    page_content = (
        f"Title: {row['Title']}, "
        f"Ingredients: {row['Ingredients']}, "
        f"Instructions: {row['Instructions']}, "
        f"Image_Name: {row['Image_Name']}, "
        f"Cleaned_Ingredients: {row['Cleaned_Ingredients']}"
    )
    
    # Create a Document object
    doc = Document(page_content=page_content)
    documents.append(doc)

print(f"Created {len(documents)} documents from the CSV file.")
# Example document content: "Product Name: Quantum Laptop, Category: Electronics, ..."


Created 13501 documents from the CSV file.


In [21]:
load_dotenv()

# Check for API keys
if not os.environ.get("PINECONE_API_KEY"):
    raise ValueError("PINECONE_API_KEY is not set in the environment variables.")

In [23]:
# This model produces 1024-dimension vectors
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-roberta-large-v1")

# --- 4. Upsert to Pinecone ---
print(f"Upserting {len(documents)} documents to Pinecone index '{index_name}'...")

# This command creates embeddings and upserts them to the Pinecone index.
# If the index does not exist, it will be created automatically.
docsearch = PineconeVectorStore.from_documents(
    documents=documents, 
    embedding=embeddings,
    index_name=index_name
)

Upserting 13501 documents to Pinecone index 'recipe-index'...


In [29]:
docsearch

In [24]:
retriever = docsearch.as_retriever()

In [27]:
query = "chicken and cheese recipies give plz"
retrieved_docs = retriever.invoke(query)

In [28]:
print(f"\n--- Query ---")
print(query)
print(f"\n--- Retrieved Documents ---")
for doc in retrieved_docs:
    print("Content:", doc.page_content)
    print("Metadata:", doc.metadata)
    print("-" * 20)


--- Query ---
chicken and cheese recipies give plz

--- Retrieved Documents ---
Content: Title: Mac and Cheese with Buffalo Chicken, Ingredients: ['2 1/4 cups all purpose flour', '1 1/2 tablespoons cayenne pepper', '1 1/2 tablespoons paprika', '1 1/2 tablespoons salt', '1 1/2 tablespoons ground cumin', '1 1/2 tablespoons ground coriander', '2 cups whole milk', '3 large eggs, beaten to blend', '4 cups cornflakes, ground to crumbs in processor', '1 pound chicken cutlets, cut crosswise into 1-inch-wide strips', 'Canola oil (for deep-frying)', '1 pound small elbow macaroni', '2 cups chopped green onions (about 8 large)', '2 tablespoons chopped fresh oregano', '8 tablespoons (1 stick) butter, divided', '3 cups chopped onions', '2 large garlic cloves, chopped', '1/2 cup all purpose flour', '4 cups whole milk', '1 pound extra-sharp cheddar cheese, coarsely grated (about 4 cups packed)', '8 ounces provolone cheese, coarsely grated (about 2 cups packed)', '2 teaspoons paprika', '1 teaspoon sal